### Q1 Data Processing (background intro)
Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`. Additionally, there are several countries with Numeric digits in their name. Make sure to remove these as well, e.g. `'Italy9'` should be `'Italy'`. 

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def answer_one():
    col_names = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    df1 = pd.read_excel('assets/Energy Indicators.xls',
                        header = 0,
                        names = col_names,
                        nrows = 227, 
                        skiprows = range(0,17),
                        usecols = range(2,6)
                            )
    df1.replace(to_replace="\.\.\.*", value=np.NaN, regex=True, inplace=True)  
    df1.replace(to_replace="\(.*\)", value="", regex=True, inplace=True) 
    df1['Country'].replace(to_replace="\d*", value="", regex=True, inplace=True)
    df1['Country'] = df1['Country'].apply(lambda x: x.strip())
    df1['Energy Supply'] = df1['Energy Supply'].apply(lambda x: x*1000000)
    rename_countries1 = {"Republic of Korea": "South Korea",
                    "United States of America": "United States",
                    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                    "China, Hong Kong Special Administrative Region": "Hong Kong"}
    def rename_country(countries_dict,x):
        if x in countries_dict.keys():
            return countries_dict[x]
        else:
            return x
    df1['Country'] = df1['Country'].apply(lambda x: rename_country(rename_countries1,x))
    Energy = df1
       
    df2 = pd.read_csv('assets/world_bank.csv',
                     header=0,
                     skiprows=(0,1,2,3))
    rename_countries2 = {"Korea, Rep.": "South Korea",
                       "Iran, Islamic Rep.": "Iran",
                       "Hong Kong SAR, China": "Hong Kong"}
    df2['Country Name'] = df2['Country Name'].apply(lambda x: rename_country(rename_countries2,x))
    GDP = df2
    
    df3 = pd.read_excel('assets/scimagojr-3.xlsx')
    ScimEn = df3
    
    Energy.set_index('Country',inplace=True)
    GDP.set_index('Country Name',inplace=True)
    ScimEn.set_index('Country',inplace=True)
    
    df_raw = pd.merge(ScimEn,Energy,how='inner',left_index=True,right_index=True)
    df_raw = pd.merge(df_raw,
         GDP[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']],
        how='inner',
        left_index=True,
        right_index=True)
    
    df = df_raw[:15]
    return df

In [ ]:
# Q2. What are the top 15 countries for average GDP over the last 10 years?
def answer_two():
    Top15 = answer_one()
    avgGDP = Top15[['2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']].mean(axis=1).rename('avgGDP').sort_values(ascending=False)
    return avgGDP

In [ ]:
# Q3. By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?
def answer_three():
    df = answer_one()
    df['span'] = df.apply(lambda x: np.subtract(x['2015'],x['2006']),axis=1)
    return df.loc[answer_two().index[5]]['span']

In [ ]:
# Q4. What country has the maximum % Renewable and what is the percentage?
def answer_four():
    df = answer_one()
    a = df['% Renewable'].sort_values(ascending=False)
    n = a.index[0]
    p = a[0]
    answer = (n,p)
    return answer

In [ ]:
# Q5. Create a new column that is the ratio of Self-Citations to Total Citations. What is the maximum value for this new column, and what country has the highest ratio?
def answer_five():
    df = answer_one()
    df['Citation Ratio'] = df['Self-citations']/df['Citations']
    mask = df['Citation Ratio'].max()
    target = df[df['Citation Ratio']==mask]
    n = target.index[0]
    r = target['Citation Ratio'][0]
    answer = (n,r)
    return answer

In [ ]:
# Q6. Create a column that estimates the population using Energy Supply and Energy Supply per capita. What is the third most populous country according to this estimate?
def answer_six():
    df = answer_one()
    df['Population'] = df['Energy Supply']/df['Energy Supply per Capita']
    target = df['Population'].sort_values(ascending=False)
    answer = target.index[2]
    return answer

In [ ]:
# Q7. Create a column that estimates the number of citable documents per person. 
def answer_seven():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    return Top15['Citable docs per Capita'].corr(Top15['Energy Supply per Capita'])    

def plot7():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])


In [ ]:
# Q8. Create a new column with a 1 if the country's % Renewable value is at or above the median 
# for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.
def answer_eight():
    df = answer_one()
    def cal_highrenew(x):
        if x >= df['% Renewable'].median():
            return 1
        else:
            return 0
    df['HighRenew'] = df['% Renewable'].apply(lambda x: cal_highrenew(x))
    df = df.sort_values('Rank',ascending=True)
    answer = df['HighRenew']
    return answer


In [ ]:
# Q9. group the Countries by Continent, then create a DataFrame that displays the sample size, sum, mean, and std deviation for the estimated population of each country
def answer_nine():
    df = answer_one()
    df['Population'] = df['Energy Supply']/df['Energy Supply per Capita']
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    df['Country']=df.index
    def get_region(x):
        if x in ContinentDict.keys():
            return ContinentDict[x]
    df['Continent'] = df['Country'].apply(lambda x: get_region(x))
    answer = df.pivot_table(values='Population',
                           index='Continent',
                           aggfunc=[len,np.sum,np.mean,np.std]
                              )
    answer.columns=('size','sum','mean','std')
    answer = answer
    return answer

In [ ]:
# Q10. Convert the Population Estimate series to a string with thousands separator (using commas). Use all significant digits
def answer_ten():
    df = answer_one()
    df['PopEst'] = df['Energy Supply']/df['Energy Supply per Capita']
    for i in range(0,15):
        df['PopEst'][i]='{:,}'.format(df['PopEst'][i])
    PopEst = pd.Series(df['PopEst'], name='PopEst')
    return PopEst

In [ ]:
# data visulization
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")